## Using real-data  

For a "real" data test case, we'll test the different aligners/search software on the virus sequences from IMG/VR4.  
In IMG/VR4  [DOI: 10.1093/nar/gkac1037](https://doi.org/10.1093/nar/gkac1037), the host assignment [workflow](https://github.com/jgi-microbiome-data-science/crispr-host-prediction) identifies spacer-protospacer pairs via `blastn -w 8 --dust no ...  --max_target_seqs 1000` which is roughly equivalent to using `blastn -task short` parameter, followed by filtering the results and using the `assign_host.py` script which utilizes the spacers' lineage information.  
Note, we do not compare any filtereing done after the pairing was done, or the host assignment itself - for that reason, we also do not benchmark dedicated host assignment tools like spacerphaser, wish or iphop (even though they may be useful or utilize spacer-protospacer pairs).  

In our benchmark, we'll use a more updated and curated version of the spacer set compared to the one used in IMG/VR4 - we'll use the CRISPR sapcer set from iphop [https://bitbucket.org/srouxjgi/iphop/src/main/#markdown-header-host-databases-and-versions](https://bitbucket.org/srouxjgi/iphop/src/main/#markdown-header-host-databases-and-versions). Specifically, we'll use the June 2025 upload.



## Fetch IMG/VR4
The full IMG/VR4 data can be downloaded from the [JGI portal](https://genome.jgi.doe.gov/portal/pages/dynamicOrganismDownload.jsf?organism=IMG_VR). This requires logining in and selecting the files we want:  
- IMGVR_all_nucleotides-high_confidence.fna
- IMGVR_all_Sequence_information-high_confidence.tsv

For convenience, we'll download the ffasta file from an unstable FTP hosting of them on nersc:
- [IMGVR4_SEQUENCES.fna.zst](https://portal.nersc.gov/genomad/__data__/IMGVR_DATA/IMGVR4_SEQUENCES.fna.zst)


In [ ]:
%%bash
mkdir imgvr4_data
cd imgvr4_data
mkdir spacers contigs

cd contigs
# download the contigs (this may take a while)
wget https://portal.nersc.gov/genomad/__data__/IMGVR_DATA/IMGVR4_SEQUENCES.fna.zst --quiet -O IMGVR4_SEQUENCES.fna.zst
# decompress the file
zstd -d IMGVR4_SEQUENCES.fna.zst
# get the name, length and GC% of the contigs (this may also take a while)
seqkit fx2tab -nl --gc IMGVR4_SEQUENCES.fna  > IMGVR4_SEQUENCES_name_length.tab

Note, we'll use the IMG_VR_2022-09-20_7.1 version of IMG/VR (v4.1, with bugfix in UViG table and protein fasta), and specifically the high-confidence genomes only ( `IMGVR_all_nucleotides-high_confidence.fna`).  
To get the info table, you'll need to login to the JGI portal and download the `IMGVR_all_Sequence_information-high_confidence.tsv` file. The data is free and unrestricted, but still requires registerition or confirming the data use agreement. 


## fetch the spacer set
We are only interested in one (or two) files from the iphop database:
- `All_CRISPR_spacers_nr_clean.fna`
- `Host_Genomes.tsv`



In [ ]:
%%bash
cd imgvr4_data/spacers
wget https://portal.nersc.gov/cfs/m342/iphop/db/extra/All_CRISPR_spacers_nr_clean.fna
wget https://portal.nersc.gov/cfs/m342/iphop/db/extra/Host_Genomes.tsv
# create a table with the name and length of the spacers
seqkit fx2tab -nl All_CRISPR_spacers_nr_clean.fna  > All_CRISPR_spacers_nr_clean_name_length.tab

Next some data wrangling to get the contig info and the spacer info in the same format.

In [ ]:
import os
os.chdir('/clusterfs/jgi/scratch/science/metagen/neri/code/blits/spacer_bench/imgvr4_data/')
import polars as pl
pl.Config(tbl_rows=50)

from bench import *
from bench.utils.functions import *

contigs_file = 'contigs/IMGVR4_SEQUENCES.fna'
spacers_file = 'spacers/All_CRISPR_spacers_nr_clean.fna'

contig_stats = pl.read_csv('contigs/IMGVR4_SEQUENCES_name_length.tab', separator='\t',has_header=False,new_columns=['seqid','length',"GC"])
contig_info = pl.read_csv('contigs/IMGVR_all_Sequence_information-high_confidence.tsv', separator='\t',infer_schema_length=10000, null_values=['N/A'])
contig_info= contig_info.rename({"Coordinates ('whole' if the UViG is the entire contig)":"coordinates"})
contig_info= contig_info.with_columns(
    pl.when(pl.col('coordinates').eq('whole')).then(None).otherwise(pl.col('coordinates')).alias('coordinates')
)
contig_info = contig_info.with_columns(
    pl.concat_str([pl.col('UVIG'),pl.col('Taxon_oid'),pl.col('Scaffold_oid'),pl.col('coordinates')],ignore_nulls=True,separator='|').alias('seqid')
)
print(f"contig_stats.schema: {contig_stats.schema}")
print(f"contig_stats.shape: {contig_stats.shape}")

print(f"contig_info.schema: {contig_info.schema}")
print(f"contig_info.shape: {contig_info.shape}")
contig_stats = contig_stats.join(contig_info, on='seqid', how='inner')

contig_stats = contig_stats.filter(pl.col("length") == pl.col("Length"))
contig_stats = contig_stats.filter(pl.col("Topology") != "GVMAG")

contig_stats = contig_stats.select(["seqid","Taxonomic classification","length","GC","Topology","MIUViG quality","UVIG"])
contig_stats = contig_stats.rename(str.lower)
contig_stats = contig_stats.rename(lambda x: x.lower().replace(" ","_"))

contig_stats = contig_stats.filter(~pl.col("taxonomic_classification").str.contains(";;;;;"))
contig_stats = contig_stats.with_columns(
    pl.when(pl.col("miuvig_quality").str.contains("High-quality|Reference")).then(True).otherwise(False).alias("hq")
)
contig_stats = contig_stats.with_columns([
    pl.col("taxonomic_classification")
    .str.extract(r"r__([^;]*)", 1).alias("realm"),
    pl.col("taxonomic_classification")
    .str.extract(r"k__([^;]*)", 1).alias("kingdom"),
    pl.col("taxonomic_classification")
    .str.extract(r"p__([^;]*)", 1).alias("phylum"),
    pl.col("taxonomic_classification")
    .str.extract(r"c__([^;]*)", 1).alias("class"),
    pl.col("taxonomic_classification")
    .str.extract(r"o__([^;]*)", 1).alias("order"),
    pl.col("taxonomic_classification")
    .str.extract(r"f__([^;]*)", 1).alias("family"),
    pl.col("taxonomic_classification")
    .str.extract(r"g__([^;]*)", 1).alias("genus"),
    pl.col("taxonomic_classification")
    .str.extract(r"s__([^;]*)", 1).alias("species"),
])
contig_stats = contig_stats.drop(["taxonomic_classification","miuvig_quality"])
for i in contig_stats.columns:
    print(f"{i}: {contig_stats[i].value_counts(sort=True)}")
# tmp.head(3)
print(f"new schema: {contig_stats.schema}")
contig_stats.head(3)

Finally, we'll index the contigs file for fast random access later.
This may take a while, but only done once and will save time (later).

In [ ]:
%%bash
pyfastx index contigs/IMGVR4_SEQUENCES.fna
ls -lsh ../imgvr4_data/contigs/IMGVR4_SEQUENCES.fna*

158G -rw-rw-r-- 1 uneri grp-org-sc-metagen 158G Sep  4  2022 ../imgvr4_data/contigs/IMGVR4_SEQUENCES.fna
2.3G -rw-r--r-- 1 uneri grp-org-sc-metagen 2.3G Oct  7 15:38 ../imgvr4_data/contigs/IMGVR4_SEQUENCES.fna.fxi


(Next steps are in the spacer inspection notebook)